#  Logistic Regression

In [8]:
t_p: read0 `:./../data/twitter_samples/positive_tweets.json
t_n: read0 `:./../data/twitter_samples/negative_tweets.json
/ t_p -> tweet positive
/ t_n -> tweet negative

In [9]:
df_positive : ((uj/) {enlist .j.k raze t_p[x]}'[til count t_p])
update sentiment:1 from `df_positive;
df_positive : select text, sentiment from df_positive;

In [10]:
df_negative:(((uj/) {enlist .j.k raze t_n[x]}'[til count t_n]))
update sentiment:0 from `df_negative;
df_negative: select text, sentiment from df_negative

In [11]:
df: df_positive uj df_negative

In [12]:
delete df_negative,df_positive,t_n,t_p from `.

`.


In [13]:
\v

,`df


In [15]:
count df

10000


In [21]:
/ Stop words
stop_words:read0 `:./../data/english

/preprocessing
stemmer:{
    suffixes : ("*ing";"*ly";"*ness";"*ed";"*ious";"*ies";"*ive";"*es";"*ment";"*s");
    if[(count x) < 3; :(x); :(::)];
    suf: suffixes where ((x like/:) suffixes);
    x: (1 - (count each suf))[0] _ x;
    :x
    };

punctuations:"!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~";

/Make word frequency dictionary
mkdic:{[d;x]$[x in key d;d[x]+:1;d[x]:1];d};

In [ ]:
fun:{[df]
    w:()!();
    v:()!();
    pos_lis: " " sv (exec text from df where sentiment = 1);
    neg_lis: " " sv (exec text from df where sentiment = 0);
    pos_lis: pos_lis except punctuations;
    neg_lis: neg_lis except punctuations;
    pos_lis:" " vs lower(pos_lis);
    neg_lis:" " vs lower(neg_lis);
    pos_lis: (pos_lis except stop_words);
    neg_lis: (neg_lis except stop_words);
    pos_lis:{.[stemmer;enlist x;x]}'[pos_lis];
    neg_lis:{.[stemmer;enlist x;x]}'[neg_lis];
    pos_lis: `$pos_lis;
    neg_lis:`$neg_lis;
    w:mkdic/[w;pos_lis];
    v:mkdic/[v;neg_lis];
    pos_t: ([word:key w]; pos:value w);
    neg_t: ([word:key v]; negv:value v);
    .Q.gc[];
    :(0^(pos_t uj neg_t))
    };

In [25]:
freqs: fun df

In [26]:
freqs

word        | pos  negv
------------| ---------
followfriday| 25   0   
franceinte  | 1    0   
pkuchly57   | 1    0   
milipolpari | 1    0   
top         | 30   5   
engag       | 7    0   
member      | 15   6   
community   | 31   1   
week        | 85   57  
            | 5277 5700
lamb2ja     | 1    0   
hey         | 75   26  
jam         | 7    5   
odd         | 2    3   
please      | 93   233 
call        | 38   29  
contact     | 7    7   
centre      | 2    1   
02392441234 | 1    0   
able        | 8    23  
..


###  Sigmoid Function
* The sigmoid function is defined as: 

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

It maps the input 'z' to a value that ranges between 0 and 1, and so it can be treated as a probability. 

In [28]:
sigmoid:{
    :(1%(1+(2.718281828459045 xexp (neg x))))
    }

## Implement gradient descent function
* The number of iterations 'num_iters" is the number of times that you'll use the entire training set.
* For each iteration, you'll calculate the cost function using all training examples (there are 'm' training examples), and for all features.
* Instead of updating a single weight $\theta_i$ at a time, we can update all the weights in the column vector:  
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\ 
\theta_2 
\\ 
\vdots
\\ 
\theta_n
\end{pmatrix}$$
* $\mathbf{\theta}$ has dimensions (n+1, 1), where 'n' is the number of features, and there is one more element for the bias term $\theta_0$ (note that the corresponding feature value $\mathbf{x_0}$ is 1).
* The 'logits', 'z', are calculated by multiplying the feature matrix 'x' with the weight vector 'theta'.  $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ has dimensions (m, n+1) 
    * $\mathbf{\theta}$: has dimensions (n+1, 1)
    * $\mathbf{z}$: has dimensions (m, 1)
* The prediction 'h', is calculated by applying the sigmoid to each element in 'z': $h(z) = sigmoid(z)$, and has dimensions (m,1).
* The cost function $J$ is calculated by taking the dot product of the vectors 'y' and 'log(h)'.  Since both 'y' and 'h' are column vectors (m,1), transpose the vector to the left, so that matrix multiplication of a row vector with column vector performs the dot product.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* The update of theta is also vectorized.  Because the dimensions of $\mathbf{x}$ are (m, n+1), and both $\mathbf{h}$ and $\mathbf{y}$ are (m, 1), we need to transpose the $\mathbf{x}$ and place it on the left in order to perform matrix multiplication, which then yields the (n+1, 1) answer we need:
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

In [123]:
gradientDescent:{[x;y;theta;alpha;num_iters] 
    m : count x;
    J : 0;
    do[num_iters;
        z : {sum theta*x}'[x];
        h : sigmoid z;
        J : (neg 1 % m) * (((sum(y * log h))) + (sum((1-y) * log (1-h))));
        theta : theta - (alpha % m)*(sum(x * (h-y)));
        show theta;
        ];
    J = "f"$J;
    :(J;theta)
    }

In [118]:
x: ((1.00000000e+00 8.34044009e+02 1.44064899e+03);(1.00000000e+00 2.28749635e-01 6.04665145e+02);(1.00000000e+00 2.93511782e+02 1.84677190e+02);(1.00000000e+00 3.72520423e+02 6.91121454e+02);(1.00000000e+00 7.93534948e+02 1.07763347e+03);(1.00000000e+00 8.38389029e+02 1.37043900e+03);(1.00000000e+00 4.08904499e+02 1.75623487e+03);(1.00000000e+00 5.47751864e+01 1.34093502e+03);(1.00000000e+00 8.34609605e+02 1.11737966e+03);(1.00000000e+00 2.80773877e+02 3.96202978e+02));
y : ((1.);(1.);(0.);(1.);(1.);(1.);(0.);(0.);(0.);(1.));

In [119]:
/gradientDescent[((1),/: (2?/: 10#10.0)*2000);(10?2);((0);(0);(0));0.000000001;700]

In [124]:
gradientDescent[x;y;((0);(0);(0));1e-8;700]

1e-09 7.63845e-07 5.907421e-07
2e-09 1.52769e-06 1.181484e-06
3e-09 2.291535e-06 1.772226e-06
4e-09 3.05538e-06 2.362969e-06
5e-09 3.819225e-06 2.953711e-06
6e-09 4.58307e-06 3.544453e-06
7e-09 5.346915e-06 4.135195e-06
8e-09 6.11076e-06 4.725937e-06
9e-09 6.874605e-06 5.316679e-06
1e-08 7.63845e-06 5.907421e-06
1.1e-08 8.402295e-06 6.498164e-06
1.2e-08 9.16614e-06 7.088906e-06
1.3e-08 9.929985e-06 7.679648e-06
1.4e-08 1.069383e-05 8.27039e-06
1.5e-08 1.145767e-05 8.861132e-06
1.6e-08 1.222152e-05 9.451874e-06
1.7e-08 1.298536e-05 1.004262e-05
1.8e-08 1.374921e-05 1.063336e-05
1.9e-08 1.451305e-05 1.12241e-05
2e-08 1.52769e-05 1.181484e-05
2.1e-08 1.604074e-05 1.240559e-05
2.2e-08 1.680459e-05 1.299633e-05
2.3e-08 1.756843e-05 1.358707e-05
2.4e-08 1.833228e-05 1.417781e-05
2.5e-08 1.909612e-05 1.476855e-05
2.6e-08 1.985997e-05 1.53593e-05
2.7e-08 2.062381e-05 1.595004e-05
2.8e-08 2.138766e-05 1.654078e-05
2.9e-08 2.21515e-05 1.713152e-05
3e-08 2.291535e-05 1.772226e-05
3.1e-08 2.367919

2.42e-07 0.0001848505 0.0001429596
2.43e-07 0.0001856143 0.0001435503
2.44e-07 0.0001863782 0.0001441411
2.45e-07 0.000187142 0.0001447318
2.46e-07 0.0001879059 0.0001453226
2.47e-07 0.0001886697 0.0001459133
2.48e-07 0.0001894336 0.0001465041
2.49e-07 0.0001901974 0.0001470948
2.5e-07 0.0001909612 0.0001476855
2.51e-07 0.0001917251 0.0001482763
2.52e-07 0.0001924889 0.000148867
2.53e-07 0.0001932528 0.0001494578
2.54e-07 0.0001940166 0.0001500485
2.55e-07 0.0001947805 0.0001506392
2.56e-07 0.0001955443 0.00015123
2.57e-07 0.0001963082 0.0001518207
2.58e-07 0.000197072 0.0001524115
2.59e-07 0.0001978359 0.0001530022
2.6e-07 0.0001985997 0.000153593
2.61e-07 0.0001993635 0.0001541837
2.62e-07 0.0002001274 0.0001547744
2.63e-07 0.0002008912 0.0001553652
2.64e-07 0.0002016551 0.0001559559
2.65e-07 0.0002024189 0.0001565467
2.66e-07 0.0002031828 0.0001571374
2.67e-07 0.0002039466 0.0001577282
2.68e-07 0.0002047105 0.0001583189
2.69e-07 0.0002054743 0.0001589096
2.7e-07 0.0002062381 0.00015

4.85e-07 0.0003704648 0.0002865099
4.86e-07 0.0003712287 0.0002871007
4.87e-07 0.0003719925 0.0002876914
4.88e-07 0.0003727564 0.0002882822
4.89e-07 0.0003735202 0.0002888729
4.9e-07 0.000374284 0.0002894637
4.91e-07 0.0003750479 0.0002900544
4.92e-07 0.0003758117 0.0002906451
4.93e-07 0.0003765756 0.0002912359
4.94e-07 0.0003773394 0.0002918266
4.95e-07 0.0003781033 0.0002924174
4.96e-07 0.0003788671 0.0002930081
4.97e-07 0.000379631 0.0002935988
4.98e-07 0.0003803948 0.0002941896
4.99e-07 0.0003811586 0.0002947803
5e-07 0.0003819225 0.0002953711
5.01e-07 0.0003826863 0.0002959618
5.02e-07 0.0003834502 0.0002965526
5.03e-07 0.000384214 0.0002971433
5.04e-07 0.0003849779 0.000297734
5.05e-07 0.0003857417 0.0002983248
5.06e-07 0.0003865056 0.0002989155
5.07e-07 0.0003872694 0.0002995063
5.08e-07 0.0003880333 0.000300097
5.09e-07 0.0003887971 0.0003006878
5.1e-07 0.0003895609 0.0003012785
5.11e-07 0.0003903248 0.0003018692
5.12e-07 0.0003910886 0.00030246
5.13e-07 0.0003918525 0.00030305

0.6931472
7e-07 0.0005346915 0.0004135195


In [125]:
theta : (0;0;0)

In [126]:
z : {sum theta*x}'[x];
h : sigmoid z;
m : count x;

In [115]:
J : (neg 1 % m) * (((sum(y * log h))) + (sum((1-y) * log (1-h))));

In [129]:
z

0 0 0 0 0 0 0 0 0 0f
